In [ ]:
import re
import pandas as pd

In [ ]:
import re
import pandas as pd

def preprocess(data, time_format=24):

    # ------------------ regex pattern ------------------
    if time_format == 12:
        pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s?(?:am|pm)\s-\s"
    else:
        pattern = r"\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s"

    messages = re.split(pattern, data)[1:]
    dates = re.findall(pattern, data)

    df = pd.DataFrame({
        "message_date": dates,
        "user_message": messages
    })

    # ------------------ datetime parsing ------------------
    if time_format == 12:
        df["date"] = pd.to_datetime(
            df["message_date"],
            format="%d/%m/%Y, %I:%M %p - ",
            dayfirst=True
        )
    else:
        df["date"] = pd.to_datetime(
            df["message_date"],
            format="%d/%m/%Y, %H:%M - ",
            dayfirst=True
        )

    # ------------------ split user & message ------------------
    users = []
    messages = []

    for message in df["user_message"]:
        entry = re.split("([\w\W]+?):\s", message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append("notification")
            messages.append(entry[0])

    df["user"] = users
    df["message"] = messages
    df.drop(columns=["user_message"], inplace=True)

    # ------------------ date features ------------------
    df["only_date"] = df["date"].dt.date
    df["year"] = df["date"].dt.year
    df["month_num"] = df["date"].dt.month
    df["month"] = df["date"].dt.month_name()
    df["day"] = df["date"].dt.day
    df["day_name"] = df["date"].dt.day_name()
    df["hour"] = df["date"].dt.hour
    df["minute"] = df["date"].dt.minute

    # ------------------ time period ------------------
    period = []
    for hour in df["hour"]:
        if hour == 23:
            period.append("23-00")
        elif hour == 0:
            period.append("00-01")
        else:
            period.append(f"{hour}-{hour+1}")

    df["period"] = period

    return df


In [ ]:
insomnia_group_chat = open("WhatsApp Chat with Insomniacs.txt", "r", encoding="utf-8")

In [ ]:
charan_chat  = open("WhatsApp Chat with Charan  Rkv.txt", "r", encoding="utf-8")

In [ ]:
fin_plan_chat = open("WhatsApp Chat with Fin Plan project.txt" , "r" , encoding="utf-8")

In [ ]:
data = fin_plan_chat.read()
data

In [ ]:
df = preprocess(data , 12)

In [ ]:
from datetime import timedelta
class TimeSlicer:
    def __init__(self , gap_minutes=30) -> None:
        self.gap_minutes = timedelta(minutes=gap_minutes)


    def slice(self, messages: list):
        if not messages:return "Bro nv manchi vaadiki kaadhu"
        slices = []
        current_slice = [messages[0]]
        for i in range(1 , len(messages)):
            prev_time = messages[i-1][2]
            curr_time = messages[i][2]
            time_diff = curr_time - prev_time
            if time_diff <= self.gap_minutes:
                current_slice.append(messages[i])
            else:
                slices.append(current_slice)
                current_slice = [messages[i]]
        if current_slice: slices.append(current_slice)
        return slices
    

In [ ]:
data = (list(zip(df["user"].to_list() , df["message"].to_list() , df["date"].to_list() )))
data[0]

In [ ]:
ts = TimeSlicer()

In [ ]:
slices = ts.slice(data)

In [ ]:
slices[0][0]

In [ ]:
def slice_text(slices):
    slice_texts = []
    for slice in slices:
        texts = [f"{msg[0] , msg[1] , msg[2]}" for msg in slice]
        slice_text = " ".join(texts)
        slice_texts.append(slice_text)
    return slice_texts
texts = slice_text(slices)

In [ ]:
metadatas = [{
            "source": f"slice_{i}",
            "start_time": str(slices[i][0][2]),
            "text": texts[i],
        } for i in range(len(texts))]

In [ ]:
from sentence_transformers import SentenceTransformer
def embed(texts):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts)
    return embeddings


In [ ]:
embeddings = embed(texts)

In [ ]:
embeddings

In [ ]:
import chromadb
class ChromaStore:
    def __init__(self) -> None:
        self.client = chromadb.Client()
        self.col = self.client.create_collection(name="finpla")
    def add_embeddings(self, embeddings, metadatas=None):
        self.col.add(
            ids = list(map(str, range(len(embeddings)))),
            embeddings=embeddings,
            metadatas=metadatas
        )
    def query(self, embedding, n_results=5):
        results = self.col.query(
            query_embeddings=[embedding],
            n_results=n_results
        )
        return results

store = ChromaStore()

In [ ]:
store.add_embeddings(embeddings ,metadatas)

In [ ]:
embeddings[13]

In [ ]:
query = list("what task we have done recently")

In [ ]:
embedq = embed(query)

In [ ]:
top_k_embeds = store.query(embedq)

In [ ]:
(top_k_embeds["metadatas"])

In [ ]:
import google.generativeai as genai
import os
genai.configure(api_key="not_stored_here")
model = genai.GenerativeModel("models/gemma-3-12b-it")

In [ ]:
prompt = "I am performing a rag based applicaiton and you are my llm agent i will provide what is the user query or question " \
         "and i will also provide you the related content which performed with embedding and getting top k related data" \
         "the user basically upload his whatsapp chat it may be group or personal chat and he/she will ask the question" \
         "so that we have to reply the answer so that user should highly satisfied and even if you don' get you say entertain the user " \
         "without giving the wrong information" \
         "you have to give a small simple answer i will directly provide the user and your answer should not include any thinking jsut a reply to user should here" \
         "don't even say i see history , i have did this , did that don't never expose the internal llm things just give the answer for quesiton and abstract the process" \
         f"here i am attaching the query =  {query} emeddings = {top_k_embeds}"


In [ ]:
response = model.generate_content(prompt)

In [ ]:
response.text